# MicroGrid Energy Management

## Summary

The goal of the Microgrid problem is to realize an optimal power flow within the distributed sources, loads, storages and a main grid. This problem can be formulated as a mixed integer linear program, for which constraints constraints, variables and objectives are organized using pyomo blocks. 

<img src="figures/mg_pv_bat_eol_house.png" width="500">

## Problem Statement

The Energy Management problem can be formulated mathematically as a mixed integer linear problem using the following model.  

### Sets
 time = ContinuousSet from 0 to H (s)

In [1]:
from pyomo.environ import *
from pyomo.dae import ContinuousSet, Integral

H = 60*60*24 # Time horizon in seconds

m = AbstractModel()
m.time = ContinuousSet(initialize=(0, H))

### Blocks

 - **Maingrid** : A block that describes the model of the distribution grid connection, a base version, named `AbsMainGridV0` is available in `microgrids.maingrids`.
 - **Renewable Power Source** : A block that describes the model of a PV panels. This will be modeled by a deterministic power profile using a `Param` indexed by the time. Such a block is available in `microgrids.sources.AbsFixedPowerSource`.  
 - **Power Load** : A block that describes the model of a critical load. This will be modeled by a deterministic power profile using a `Param` indexed by the time. Such a block is available in `microgrids.sources.AbsFixedPowerLoad`.  

In [2]:
from batteries import AbsBatteryV0
from maingrids import AbsMainGridV0
from sources import AbsFixedPowerLoad, AbsFixedPowerSource

m.mg = AbsMainGridV0()
m.s  = AbsFixedPowerSource()
m.l  = AbsFixedPowerLoad()

One can print any pyomo object using the `pprint` method. Example : 
    
    m.mg.pprint()
One can access documentation of any object using the builtIn method `doc` or `help` function (for heritance). Pop-up documentation shortcut : `Shift+Tab`.

    print(m.mg.doc)
    help(m.mg)

In [3]:
@m.Constraint(m.time)
def power_node(m, t):
    return m.mg.p[t] + m.s.p[t] == m.l.p[t]

In [4]:
m.int = Integral(m.time, wrt=m.time, rule=lambda m, i: m.mg.inst_cost[i])

m.obj = Objective(expr=m.int)

In [5]:
%run data/data_models.py
df_s[['P_pv', 'P_load_1']].plot(figsize=(15,3))

/home/admin/anaconda3/envs/pyomogallery/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [6]:
inst = m.create_instance(data)
inst.time.pprint()

time : Dim=0, Dimen=1, Size=2, Domain=None, Ordered=Sorted, Bounds=(0.0, 86400.0)
    [0.0, 86400.0]


In [7]:
from pyomo.environ import TransformationFactory
inst = m.create_instance(data)

nfe = 60*60*24/(10*60)
TransformationFactory('dae.finite_difference').apply_to(inst, nfe=nfe)